In [1]:
import findspark
findspark.init()
import pyspark
import os
import pandas as pd
import geohash

sc = pyspark.SparkContext()
spark = pyspark.SQLContext(sc)

working_dir = os.getcwd()

First, we read the data processed in the notebook *"data-cleansing.ipynb"*

In [2]:
#processedDf = pd.read_csv('processed_dataset.csv')
# Remove first column which contains a no needed index
#processedDf = processedDf.drop(processedDf.columns[0], axis=1)
# Remove eventTimeEnd because we won't use it
#processedDf = processedDf.drop('eventTimeEnd', axis=1)
#processedDf.head(10)
# Load parquet file into dataframe
processedDf = spark.read.csv("file://" + working_dir + "/processed-dataset.csv", header=True)
processedDf = processedDf.select(['eventTimeStart', 'latStart','lonStart', 'latEnd', 'lonEnd'])
processedDf.show(10, False)

+-------------------+----------+---------+---------+---------+
|eventTimeStart     |latStart  |lonStart |latEnd   |lonEnd   |
+-------------------+----------+---------+---------+---------+
|2017-03-22 16:50:25|38.7735368|-9.168737|38.76522 |-9.098054|
|2017-05-24 12:20:56|47.409291 |8.546942 |47.423743|8.555213 |
|2017-05-24 12:21:08|47.409291 |8.546942 |47.423743|8.555213 |
|2017-05-24 12:21:17|47.409291 |8.546942 |47.423743|8.555213 |
|2017-05-24 12:21:24|47.409291 |8.546942 |47.423743|8.555213 |
|2017-05-24 12:21:32|47.409291 |8.546942 |47.423743|8.555213 |
|2017-05-24 12:21:37|47.409291 |8.546942 |47.423743|8.555213 |
|2017-05-24 12:22:08|47.409291 |8.546942 |47.423743|8.555213 |
|2017-05-24 12:22:23|47.409291 |8.546942 |47.423743|8.555213 |
|2017-05-24 12:22:27|47.409291 |8.546942 |47.423743|8.555213 |
+-------------------+----------+---------+---------+---------+
only showing top 10 rows



# Data preparation specification

Given, a certain granularity in location (geohash length g), granularity in time (bins per day b) and a chosen wideness (w) of the neighbourhood we want to look at, the aggregated data in the end should have the following columns:

**"geohash"**

Geohash with length g (categorical feature). This column will not actually be used in the prediction. It is just an id and can be used when calculating the distances between the geohashes.

**"time_cat"**

Time of the day as a categorical feature. If $b = 24$ (one bin for every hour), then "time_cat" for a pickup at 14:20:00 should be the string "14:00". If $b = 96$ (one bin for every quarter of an hour), then "time_cat" for a pickup at 14:20:00 should be the string '14:15'.

**"time_num"**

Time of the day as a (binned!) floating point number between 0 and 1, where the center of the bin is converted to a floating point number between 0 and 1. So if $b = 24$, then "time_num" for a pickup at 14:20:00 should be $14.5\,/\,24 =  0.6042$. If $b = 96$, it should translate to $14.375\,/\,24 = 0.5990$.

**"time_cos"**

The binned "time_num" variable converted to a cosine version so that time nicely 'loops' rather than going saw-like when it traverses midnight. See the figure below. This transformation doesn't have any magic powers, but it can make it easier for a model to find the right patterns. "time_cos" = $\cos(\textrm{time_num} \cdot 2\pi)$. So for 24 bins, 14:20:00 would translate to $\cos(0.6042 \cdot 2\pi) = -0.7932$.

[cyclic-transf]: ./images/cyclic-numeric-feature-transformation.png
![alt text][cyclic-transf]

**"time_sin"**

Same thing as 4) but then with sine. So, "time_sin" = $\sin(\textrm{time_num} \cdot 2 \pi)$. For 24 bins per day, 14:20:00 would translate to $\sin(0.6042 \cdot 2 \pi) = -0.6089$.

**"day_cat"**

Day of the week as a categorical feature: "Monday", "Tuesday", etc.

**"day_num"**

Day of the week as a numerical feature going from 0 (Monday morning, start of the week) to 1 (Sunday night), European style. With 24 bins, Tuesday afternoon 14:20:00 would translate to $(1 + \frac{14.5}{24})\,/\,7 = 0.2292$.

**"day_cos"**

Binned "day_num" variable converted to a cosine version. "day_cos" = $\cos(\textrm{day_num} \cdot 2\pi)$

**"day_sin"**

Binned "day_num"variable converted to a sine version. "day_sin" = $\sin(\textrm{day_num} \cdot 2\pi)$

**"weekend"**

0 if weekday, 1 if weekend (Saturday/Sunday)

**Location features**

Latitude and longitude of the center of the geohash the record was bucketed in.

#### Helper functions for cleaning and feature extraction/generation

In [3]:
# Needed libraries
import time
from datetime import date
import math

def date_extractor(date_str,b,minutes_per_bin):
    # Takes a datetime object as a parameter
    # and extracts and returns a tuple of the form: (as per the data specification)
    # (time_cat, time_num, time_cos, time_sin, day_cat, day_num, day_cos, day_sin, weekend)
    # Split date string into list of date, time
    
    d = date_str.split()
    
    #safety check
    if len(d) != 2:
        return tuple([None,])
    
    # TIME (eg. for 16:56:20 and 15 mins per bin)
    #list of hour,min,sec (e.g. [16,56,20])
    time_list = [int(t) for t in d[1].split(':')]
    
    #safety check
    if len(time_list) != 3:
        return tuple([None,])
    
    # calculate number of minute into the day (eg. 1016)
    num_minutes = time_list[0] * 60 + time_list[1]
    
    # Time of the start of the bin
    time_bin = num_minutes / minutes_per_bin     # eg. 1005
    hour_bin = num_minutes / 60                  # eg. 16
    min_bin = (time_bin * minutes_per_bin) % 60  # eg. 45
    
    #get time_cat
    hour_str = str(hour_bin) if hour_bin / 10 > 0 else "0" + str(hour_bin)  # eg. "16"
    min_str = str(min_bin) if min_bin / 10 > 0 else "0" + str(min_bin)      # eg. "45"
    time_cat = hour_str + ":" + min_str                                     # eg. "16:45"
    
    # Get a floating point representation of the center of the time bin
    time_num = (hour_bin*60 + min_bin + minutes_per_bin / 2.0)/(60*24)      # eg. 0.7065972222222222
    
    time_cos = math.cos(time_num * 2 * math.pi)
    time_sin = math.sin(time_num * 2 * math.pi)
    
    # DATE
    # Parse year, month, day
    date_list = d[0].split('-')
    d_obj = date(int(date_list[0]),int(date_list[1]),int(date_list[2]))
    day_to_str = {0: "Monday",
                  1: "Tuesday",
                  2: "Wednesday",
                  3: "Thursday",
                  4: "Friday",
                  5: "Saturday",
                  6: "Sunday"}
    day_of_week = d_obj.weekday()
    day_cat = day_to_str[day_of_week]
    day_num = (day_of_week + time_num)/7.0
    day_cos = math.cos(day_num * 2 * math.pi)
    day_sin = math.sin(day_num * 2 * math.pi)
    
    year = d_obj.year
    month = d_obj.month
    day = d_obj.day
    
    weekend = 0
    #check if it is the weekend
    if day_of_week in [5,6]:
        weekend = 1
       
    return (year, month, day, time_cat, time_num, time_cos, time_sin, day_cat, day_num, day_cos, day_sin, weekend)

In [4]:
def data_cleaner(zipped_row):
    # takes a tuple (row,g,b,minutes_per_bin) as a parameter and returns a tuple of the form:
    # (time_cat, time_num, time_cos, time_sin, day_cat, day_num, day_cos, day_sin, weekend,geohash)
    row = zipped_row[0]
    g = zipped_row[1]
    b = zipped_row[2]
    minutes_per_bin = zipped_row[3]
    # The indices of trip-start datetime, latitude start, longitude start, latitude end and longitude end respectively
    indices = (0, 1, 2, 3, 4)
    
    #safety check: make sure row has enough features
    if len(row) < 5:
        return None
    
    #extract day of the week and hour
    date_str = row[indices[0]]
    clean_date = date_extractor(date_str,b,minutes_per_bin)
    #get geo hash

    lat_start = float(row[indices[1]])
    lon_start = float(row[indices[2]])
    lat_end = float(row[indices[3]])
    lon_end = float(row[indices[4]])
    location_start = None
    location_end = None
    location_start = geohash.encode(lat_start,lon_start, g)    
    location_end = geohash.encode(lat_end,lon_end, g)
    x_start = math.cos(lat_start) * math.cos(lon_start)
    y_start = math.cos(lat_start) * math.sin(lon_start) 
    z_start = math.sin(lat_start) 

    return tuple(list(clean_date)+[x_start]+[y_start]+[z_start]+[location_start]+[location_end])

#### Specify Parameters

In [5]:
g = 9 #geohash length, a 1.2km x 609.4m square area
b = 12 # number of time bins per day
# Note: b must evenly divide 60
minutes_per_bin = int((24 / float(b)) * 60)

#### Clean data create and create features as specified above

In [6]:
processedRDD = processedDf.rdd
processedRDD = processedRDD \
                .map(lambda row: (row, g, b, minutes_per_bin)) \
                .map(data_cleaner) \
                .filter(lambda row: row != None)

In [7]:
featuredDf = spark.createDataFrame(processedRDD, ['year', 'month', 'day', 'time_cat', 'time_num', 'time_cos', \
                                                  'time_sin', 'day_cat', 'day_num', 'day_cos', 'day_sin', 'weekend', \
                                                  'x_start', 'y_start', 'z_start','location_start', 'location_end'])

In [8]:
featuredPandas = featuredDf.toPandas()
featuredPandas.head(10)

,year,month,day,time_cat,time_num,time_cos,time_sin,day_cat,day_num,day_cos,day_sin,weekend,x_start,y_start,z_start,location_start,location_end
0,2017,3,22,16.833333333333332:49.999999999999886,0.777778,0.173648,-0.984808,Wednesday,0.396825,-0.797133,0.603804,0,-0.460713,-0.120608,0.879316,eyckx6e80,eycs8byzz
1,2017,5,24,12.333333333333334:20.0,0.569444,-0.906308,-0.422618,Wednesday,0.367063,-0.670981,0.741474,0,0.612977,-0.738237,-0.281542,u0qjdqxen,u0qjf9jxq
2,2017,5,24,12.35:21.0,0.570833,-0.902585,-0.430511,Wednesday,0.367262,-0.671905,0.740637,0,0.612977,-0.738237,-0.281542,u0qjdqxen,u0qjf9jxq
3,2017,5,24,12.35:21.0,0.570833,-0.902585,-0.430511,Wednesday,0.367262,-0.671905,0.740637,0,0.612977,-0.738237,-0.281542,u0qjdqxen,u0qjf9jxq
4,2017,5,24,12.35:21.0,0.570833,-0.902585,-0.430511,Wednesday,0.367262,-0.671905,0.740637,0,0.612977,-0.738237,-0.281542,u0qjdqxen,u0qjf9jxq
5,2017,5,24,12.35:21.0,0.570833,-0.902585,-0.430511,Wednesday,0.367262,-0.671905,0.740637,0,0.612977,-0.738237,-0.281542,u0qjdqxen,u0qjf9jxq
6,2017,5,24,12.35:21.0,0.570833,-0.902585,-0.430511,Wednesday,0.367262,-0.671905,0.740637,0,0.612977,-0.738237,-0.281542,u0qjdqxen,u0qjf9jxq
7,2017,5,24,12.366666666666667:22.0,0.572222,-0.898794,-0.438371,Wednesday,0.367460,-0.672828,0.739799,0,0.612977,-0.738237,-0.281542,u0qjdqxen,u0qjf9jxq
8,2017,5,24,12.366666666666667:22.0,0.572222,-0.898794,-0.438371,Wednesday,0.367460,-0.672828,0.739799,0,0.612977,-0.738237,-0.281542,u0qjdqxen,u0qjf9jxq
9,2017,5,24,12.366666666666667:22.0,0.572222,-0.898794,-0.438371,Wednesday,0.367460,-0.672828,0.739799,0,0.612977,-0.738237,-0.281542,u0qjdqxen,u0qjf9jxq


In [9]:
featuredPandas.to_csv('featured-dataset.csv')